## Starbucks existence in the cities
The main purpose of this Notebook is to query Foursquare for Starbucks venues in every city of our previously prepared dataset. The cities data was prepared Capstone_Data.ipynb and exported to csv file.

If the response received from Foursqaure is empty, we consider the city not to have Starbucks location, thus we exclude it from the list of candidate cities for the next released Starbucks mug.

In [1]:
# import 
import pandas as pd
import numpy as np
import requests
import folium

from pandas.io.json import json_normalize

!pip install lxml
import lxml

!pip install geopy
from geopy.geocoders import Nominatim

     |████████████████████████████████| 5.8MB 34.7MB/s eta 0:00:01
     |████████████████████████████████| 102kB 23.7MB/s ta 0:00:01


### Testing the idea
To test how the idea works, I will query Foursquare for Starbuckses in my home city of Yekaterinburg, Russia

In [3]:
# Get geolocation data for the city
address = 'Yekaterinburg, Russia'

geolocator = Nominatim(user_agent="city_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('Geograpical coordinates are {}, {}.'.format(latitude, longitude))
location

Geograpical coordinates are 56.839104, 60.60825.


Location(Екатеринбург, городской округ Екатеринбург, Свердловская область, Уральский федеральный округ, Россия, (56.839104, 60.60825, 0.0))

In [4]:
# define Foursquare access requisites
CLIENT_ID = 'MXKZ23XI01RX5PKFPBHKCFIR2HTZIOLZ3XXDKKH1INS0I544' # your Foursquare ID
CLIENT_SECRET = 'AQFNOIYVJLICUIY1KE22HHYC3WKIDRHH14D2FPATYVLJ4ABI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: MXKZ23XI01RX5PKFPBHKCFIR2HTZIOLZ3XXDKKH1INS0I544
CLIENT_SECRET:AQFNOIYVJLICUIY1KE22HHYC3WKIDRHH14D2FPATYVLJ4ABI


In [5]:
# create Yekaterinburg with Folium map using latitude and longitude values
map_home = folium.Map(location=[latitude, longitude], zoom_start=14)

folium.CircleMarker(
        [latitude, longitude],
        radius=8,
        popup='Home',
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_home)  

map_home

In [9]:
# prepare the URL to query Foursquare for Starbucks location in Yekaterinburg
radius = 5000
limit = 50
name = 'Starbucks'
intent = ''

url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query={}&intent={}&limit={}&radius={}".format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    latitude,
    longitude,
    name,
    intent,
    limit,
    radius)
print(url)

https://api.foursquare.com/v2/venues/search?client_id=MXKZ23XI01RX5PKFPBHKCFIR2HTZIOLZ3XXDKKH1INS0I544&client_secret=AQFNOIYVJLICUIY1KE22HHYC3WKIDRHH14D2FPATYVLJ4ABI&v=20180605&ll=56.839104,60.60825&query=Starbucks&intent=&limit=50&radius=5000


In [10]:
# send request
results = requests.get(url).json()

In [15]:
# parse response
shops_result = results['response']['venues']
shops = json_normalize(shops_result) # flatten JSON
shops.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,541accc2498ec5231d018dad,Starbucks (Старбакс),"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1571100774,False,ТРЦ «Гринвич»,"ул. 8 Марта, 46",56.828215,60.599125,"[{'label': 'display', 'lat': 56.82821451481234...",1333,620144,RU,Екатеринбург,Свердловская обл.,Россия,"[ТРЦ «Гринвич» (ул. 8 Марта, 46), 620144, Екат..."
1,59e20e1a92e7a93ccacd0d62,Starbucks,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1571100774,False,улица Ленина,улица Вайнера,56.836621,60.595589,"[{'label': 'display', 'lat': 56.83662098157948...",819,NaN,RU,Екатеринбург,Свердловская обл.,Россия,"[улица Ленина (улица Вайнера), Екатеринбург, Р..."


Great! Response is not empty, it tells there are at least two Starbucks shops in Yekaterinburg, which quite right!

In [16]:
# drop the columns we don't need
filtered_columns = ['name', 'location.address', 'location.lat', 'location.lng']
shops = shops.loc[:, filtered_columns]
shops.columns = ['name', 'address', 'latitude', 'longitude']
shops.head()

,name,address,latitude,longitude
0,Starbucks (Старбакс),ТРЦ «Гринвич»,56.828215,60.599125
1,Starbucks,улица Ленина,56.836621,60.595589


In [17]:
# this kind of a test expression we will use to tell if city has Starbucks location or not
shops.loc[shops.name.str.contains('Starbucks')].empty

False

False: Not empty, meaning it has a Starbucks shop

In [21]:
# Show Starbucks markers on Yekaterinburg's map
for lat, lng, name, address in zip(shops['latitude'], shops['longitude'], shops['name'], shops['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#31cc88',
        fill_opacity=0.7,
        parse_html=False).add_to(map_home)  
    
map_home

Exactly where they are!

### Querying Foursquare
Since Foursquare has a quota limit on requests per hour, I had to go through the data set in several runs, saving the middle results in csv file. The final version of the file is c3_full.csv

In [23]:
# Open original and most recent versions of the file
c3 = pd.read_csv('c3.csv')
c35 = pd.read_csv('c3_full.csv')

In [24]:
# check how many lines left to query
c35.loc[c35.hasShop.isnull()].shape

(0, 9)

In [26]:
# show the lines where the prevoius cycle had ended
# some of hasShop values used to be NaN, that's where I would start the mext cycle
# the final version does not have NaN values, which was the goal of the whole excrecise
c35.head(11002).tail()

,city,country,geonameid,latitude,longitude,country code,population,hasMug,hasShop
10997,Silkeborg,Denmark,2614030,56.16970,9.54508,DK,41674,False,False
10998,Sillod,India,1256269,20.30303,75.65284,IN,51042,False,False
10999,Silopi,Turkey,300797,37.24379,42.46345,TR,74798,False,True
11000,Silvan,Turkey,300796,38.13708,41.00817,TR,65956,False,False
11001,Silver Lake,United States,10104153,34.08668,-118.27023,US,32890,False,True


In [27]:
c35.shape

(13378, 9)

In [28]:
ch = c3.head().reset_index()

In [29]:
c3[c3['country']=='Algeria']

,city,country,geonameid,latitude,longitude,country code,population,hasMug
0,'Ain Benian,Algeria,2508275,36.80277,2.92185,DZ,31102,False
1,'Ain Deheb,Algeria,2508225,34.84218,1.54697,DZ,36146,False
2,'Ain Merane,Algeria,2508010,36.16277,0.97037,DZ,34413,False
3,'Ain el Bell,Algeria,2508184,34.34381,3.22475,DZ,31916,False
4,'Ain el Melh,Algeria,2508130,34.84146,4.16383,DZ,39798,False
...,...,...,...,...,...,...,...,...
11934,Tlemcen,Algeria,2475687,34.87833,-1.31500,DZ,132341,False
11960,Tolga,Algeria,2475612,34.72224,5.37845,DZ,68246,False
12021,Touggourt,Algeria,2475475,33.10527,6.05796,DZ,49044,False
13361,`Ain el Hadjel,Algeria,2508157,35.67003,3.88153,DZ,42527,False


In [9]:
# the for loop that queryies Foursquare and saves results whenever index is divided by 1000, i.e. after every 1000 response

radius = 500
limit = 50
name = 'Starbucks'
intent = ''

for index, row in c35.iterrows():
     if index > 11000: 
        latitude = row['latitude']
        longitude = row['longitude']
        
        url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query={}&intent={}&limit={}&radius={}".format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            latitude,
            longitude,
            name,
            intent,
            limit,
            radius)      
    
        results = requests.get(url).json()
        shops_result = results['response']['venues']
        shops = json_normalize(shops_result) # flatten JSON
        
        if shops.empty: 
            c35.at[index,'hasShop'] = False
        else:
            c35.at[index,'hasShop'] = not (shops.loc[shops.name.str.contains('Starbucks')].empty)
        
        if index % 1000 == 0:
            c35.to_csv('c3' + '_' + str(index) + '.csv', index = False)
            
            
    #print(c3.at[index,'city'],c3.at[index,'hasShop'])
    
print('Done')

Done


In [12]:
# save results to .csv for futire reference
c35.to_csv('c3' + '_' + 'full.csv', index = False)

In [16]:
# that's what response looks like when the quota is exceeded
results

{'meta': {'code': 403,
  'errorType': 'rate_limit_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5d9ff2b5446ea6002c3a53d3'},
 'response': {}}

In [35]:
# now that we have full dataset with Starbucks locations info, let's look at it. cf is for cities_full
cf = c35

In [31]:
cf.head()

,city,country,geonameid,latitude,longitude,country code,population,hasMug,hasShop
0,'Ain Benian,Algeria,2508275,36.80277,2.92185,DZ,31102,False,False
1,'Ain Deheb,Algeria,2508225,34.84218,1.54697,DZ,36146,False,False
2,'Ain Merane,Algeria,2508010,36.16277,0.97037,DZ,34413,False,False
3,'Ain el Bell,Algeria,2508184,34.34381,3.22475,DZ,31916,False,False
4,'Ain el Melh,Algeria,2508130,34.84146,4.16383,DZ,39798,False,False


### Query Results
Here is what we were looking for, the cities of the world, that have Starbucks stores but do not have Starbucks mugs, sorted by population in descending order

In [34]:
# grouped list - only one city from each country
cities_list = cf.loc[cf.hasShop].loc[cf.hasMug==False].sort_values('population', ascending=False).groupby('country').first().sort_values('population', ascending=False)
cities_list.shape

(107, 8)

In [36]:
# top 20
cities_list.head(20)

,city,geonameid,latitude,longitude,country code,population,hasMug,hasShop
country,,,,,,,,
India,Delhi,1273294,28.65195,77.23149,IN,10927986,False,True
Egypt,Alexandria,361058,31.21564,29.95527,EG,3811516,False,True
Japan,Yokohama,1848354,35.43333,139.65000,JP,3574443,False,True
China,Shiyan,1794903,32.64750,110.77806,CN,3460000,False,True
South Africa,Cape Town,3369157,-33.92584,18.42322,ZA,3433441,False,True
South Korea,Incheon,1843564,37.45646,126.70515,KR,2628000,False,True
Brazil,Fortaleza,3399415,-3.71722,-38.54306,BR,2400000,False,True
Indonesia,Surabaya,1625822,-7.24917,112.75083,ID,2374658,False,True
Italy,Rome,3169070,41.89193,12.51133,IT,2318895,False,True


In [37]:
# full list - not limited to 1 from each country
cities_full = cf.loc[cf.hasShop].loc[cf.hasMug==False].sort_values('population', ascending=False).sort_values('population', ascending=False)
cities_full.shape

(3956, 9)

In [38]:
# top 20
cities_full.head(20)

,city,country,geonameid,latitude,longitude,country code,population,hasMug,hasShop
3066,Delhi,India,1273294,28.65195,77.23149,IN,10927986,False,True
1339,Bengaluru,India,1277333,12.97194,77.59369,IN,5104047,False,True
6009,Kolkata,India,1275004,22.56263,88.36304,IN,4631392,False,True
2415,Chennai,India,1264527,13.08784,80.27847,IN,4328063,False,True
337,Alexandria,Egypt,361058,31.21564,29.95527,EG,3811516,False,True
150,Ahmedabad,India,1279233,23.02579,72.58727,IN,3719710,False,True
4811,Hyderabad,India,1269843,17.38405,78.45636,IN,3597816,False,True
13127,Yokohama,Japan,1848354,35.43333,139.65000,JP,3574443,False,True
10902,Shiyan,China,1794903,32.64750,110.77806,CN,3460000,False,True
2138,Cape Town,South Africa,3369157,-33.92584,18.42322,ZA,3433441,False,True


### Next 
We can play around with this data as long as we like.
Just for the sake of diversity, for the rest of this research we will stick to the list of cities limited to one from each country.
We will also drop Delhi from this list, since there is a Starbucks mug "New Delhi". But Delhi appears in the UN dataset twice - as Delhi and as New Delhi.

![New Delhi](new_delhi.jpg)

Quick data insight: as soon as we drop out Delhi, all cities are less than 10 million in population. Which means, every city in the world that has Starbucks and is more than 10 million in population does have Starbucks mug!

We will continue the research in Capstone_KNN notebook 